In [5]:
import pandas as pd
import string
import nltk

In [6]:
Data=pd.read_csv("train.csv",sep= "~")

In [7]:
Data.head(20)

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,11755,After reading mixed reviews I almost didn't bo...,Google Chrome,Desktop,Good
1,33912,This motor inn is located about - city blocks ...,Firefox,Tablet,Good
2,10143,It was our first time there and surely not our...,Google Chrome,Mobile,Good
3,33114,"Great hotel in an excellent location, just off...",Mozilla,Desktop,Good
4,17464,We stayed at the hotel for - weeks to get away...,Google Chrome,Desktop,Good
5,34367,Myself and two girlfriends were in NYC for - n...,InternetExplorer,Mobile,Good
6,14524,I made last minute reservation and couldnt fin...,Edge,Desktop,Bad
7,35130,Stayed at this hotel for - week. Very nice hot...,InternetExplorer,Mobile,Bad
8,1004,My wife and I stayed in the Michelangelo for -...,InternetExplorer,Desktop,Good
9,27086,My wife and I stayed their after a business me...,InternetExplorer,Desktop,Good


In [8]:
Data.shape

(30172, 5)

In [9]:
Data.isna().sum()

User_ID         0
Description     0
Browser_Used    0
Device_Used     0
Is_Response     0
dtype: int64

In [10]:
Data.describe(include="all")

,User_ID,Description,Browser_Used,Device_Used,Is_Response
count,30172.000000,30172,30172,30172,30172
unique,NaN,30172,11,3,2
top,NaN,"I've stayed at many Comfort Inns, and reviewed...",Firefox,Mobile,Good
freq,NaN,1,5754,11639,20567
mean,19394.643113,NaN,NaN,NaN,NaN
std,11255.034590,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN
25%,9649.750000,NaN,NaN,NaN,NaN
50%,19358.500000,NaN,NaN,NaN,NaN
75%,29173.250000,NaN,NaN,NaN,NaN


In [11]:
Data=Data.drop(["Browser_Used","Device_Used"],axis=1)  #Dropping Browser used and Device Used as it not Relevant We can use 
                                                       #it for grouping thogh to see from which browser most Response is 
                                                       #Coming using which Device

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
label=LabelEncoder()
Data['Categorial Response']=label.fit_transform(Data["Is_Response"])

In [14]:
Data=Data.drop(['Is_Response'],axis=1)

In [15]:
Data.head()

,User_ID,Description,Categorial Response
0,11755,After reading mixed reviews I almost didn't bo...,1
1,33912,This motor inn is located about - city blocks ...,1
2,10143,It was our first time there and surely not our...,1
3,33114,"Great hotel in an excellent location, just off...",1
4,17464,We stayed at the hotel for - weeks to get away...,1


# Cleaning the Data

In [16]:
from nltk.corpus import stopwords
stopwords_list = set(stopwords.words("english"))
stopwords_list.add('.')
stopwords_list.add(':')  
stopwords_list.add(',') 
stopwords_list.add(';')
stopwords_list.add('&')
stopwords_list.add("'")
stopwords_list.add("-")


In [17]:
Data.Description = Data.Description.apply(lambda x: x.lower())
Data.Description = Data.Description.apply(lambda x: x.translate(string.punctuation))
Data.Description = Data.Description.apply(lambda x: x.translate(string.digits))

In [18]:
Data.head()

,User_ID,Description,Categorial Response
0,11755,after reading mixed reviews i almost didn't bo...,1
1,33912,this motor inn is located about - city blocks ...,1
2,10143,it was our first time there and surely not our...,1
3,33114,"great hotel in an excellent location, just off...",1
4,17464,we stayed at the hotel for - weeks to get away...,1


In [19]:
Data['Description_without_stopwords'] = Data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))


In [20]:
def stemmer(text):
        ps=nltk.porter.PorterStemmer()
        tokens = text.split()
        stemmed_tokens = [ps.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
Data['Clean_Description'] = Data['Description_without_stopwords'].apply(stemmer)    

In [21]:
Data=Data.drop(['Description_without_stopwords','Description'],axis=1)

# Model Building

In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

In [24]:
#Function to call any Algorithm
def evaluate_features(X, y, clf=None):
    if clf is None:
        clf = LogisticRegression()
    
    probas = cross_val_predict(clf, X, y, cv=KFold(random_state=2019,n_splits=6), 
                              n_jobs=-1, method='predict_proba')
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    print('Classification report',classification_report(y,preds))
    print('Confusion Matrix  ', confusion_matrix(y,preds))

In [25]:
count_vectorizer = CountVectorizer(max_features=45000,ngram_range=(1,2))  

bag_of_words = count_vectorizer.fit_transform(Data["Clean_Description"])
    
len(count_vectorizer.get_feature_names())

45000

In [29]:
evaluate_features(bag_of_words, Data['Categorial Response'])

Log loss: 0.3989687709405552
Accuracy: 0.8747514251624022
Classification report               precision    recall  f1-score   support

           0       0.83      0.77      0.80      9605
           1       0.89      0.92      0.91     20567

   micro avg       0.87      0.87      0.87     30172
   macro avg       0.86      0.85      0.85     30172
weighted avg       0.87      0.87      0.87     30172

Confusion Matrix   [[ 7371  2234]
 [ 1545 19022]]


In [ ]:
#evaluate_features(bag_of_words, Data['Categorial Response'],SVC(kernel='linear',probability=True,random_state=2019))

In [30]:
tfidf_vectorizer = TfidfVectorizer(max_features=45000,ngram_range=(1,2),analyzer='word')    

tfidf = tfidf_vectorizer.fit_transform(Data['Clean_Description'])

len(count_vectorizer.get_feature_names())

45000

In [31]:
evaluate_features(tfidf, Data['Categorial Response'])

Log loss: 0.3038463562318182
Accuracy: 0.8841309823677582
Classification report               precision    recall  f1-score   support

           0       0.88      0.74      0.80      9605
           1       0.89      0.95      0.92     20567

   micro avg       0.88      0.88      0.88     30172
   macro avg       0.88      0.85      0.86     30172
weighted avg       0.88      0.88      0.88     30172

Confusion Matrix   [[ 7126  2479]
 [ 1017 19550]]


# We can see that decreasing features is not affecting much on Accuracy Score We can decrease our Features more to Speed Up the Process

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
evaluate_features(tfidf,Data['Categorial Response'],MultinomialNB())

Log loss: 0.3253999225550261
Accuracy: 0.8584117725043087
Classification report               precision    recall  f1-score   support

           0       0.91      0.62      0.73      9605
           1       0.84      0.97      0.90     20567

   micro avg       0.86      0.86      0.86     30172
   macro avg       0.88      0.79      0.82     30172
weighted avg       0.87      0.86      0.85     30172

Confusion Matrix   [[ 5912  3693]
 [  579 19988]]


In [38]:
#We can also use Xgboost but it will increase our computational cost and will not give much difference in Accuracy.
#We can also do Topic Modeling.

# Test Data

In [26]:
DataTest=pd.read_csv("test.csv",sep="~")
DataTest.head()

,User_ID,Description,Browser_Used,Device_Used
0,9602,A friend and I stayed in this hotel when we we...,Edge,Desktop
1,8749,I enjoy staying here when I have early flights...,Google Chrome,Mobile
2,15500,I stopped off in Seattle during a train tour o...,Chrome,Mobile
3,5495,I have stayed at this hotel - or - times now f...,Mozilla Firefox,Desktop
4,18570,Excellent location with hop on hop off city tr...,Edge,Mobile


In [27]:
DataTest.Description = DataTest.Description.apply(lambda x: x.lower())
DataTest.Description = DataTest.Description.apply(lambda x: x.translate(string.punctuation))
DataTest.Description = DataTest.Description.apply(lambda x: x.translate(string.digits))

In [28]:
DataTest['Description_without_stopwords'] = DataTest['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))


In [29]:
def stemmer(text):
        ps=nltk.porter.PorterStemmer()
        tokens = text.split()
        stemmed_tokens = [ps.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
DataTest['Clean_Description'] = DataTest['Description_without_stopwords'].apply(stemmer)    

In [30]:
DataTest=DataTest.drop(['Description_without_stopwords','Description'],axis=1)

In [31]:
X=Data["Clean_Description"]
y=Data.drop(['Clean_Description','User_ID'],axis=1)

In [32]:
y.head()

,Categorial Response
0,1
1,1
2,1
3,1
4,1


In [33]:
from sklearn.model_selection import KFold

In [34]:
kf=KFold(n_splits=5,shuffle=False,random_state=55)
for train_index,test_index in kf.split(X):
    print("Train",train_index,"Validation",test_index)
    X_train,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]

Train [ 6035  6036  6037 ... 30169 30170 30171] Validation [   0    1    2 ... 6032 6033 6034]
Train [    0     1     2 ... 30169 30170 30171] Validation [ 6035  6036  6037 ... 12067 12068 12069]
Train [    0     1     2 ... 30169 30170 30171] Validation [12070 12071 12072 ... 18101 18102 18103]
Train [    0     1     2 ... 30169 30170 30171] Validation [18104 18105 18106 ... 24135 24136 24137]
Train [    0     1     2 ... 24135 24136 24137] Validation [24138 24139 24140 ... 30169 30170 30171]


In [35]:
basictf= TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)
basicTrain=basictf.fit_transform(X_train)
print(basicTrain.shape)

(24138, 818994)


In [36]:
basicModel=LogisticRegression()
basicModel = basicModel.fit(basicTrain, y_train)
basicTest = basictf.transform(X_test)
y_pred = basicModel.predict(basicTest)

C:\Users\sagar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sagar\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
print(confusion_matrix(y_test, y_pred))
print (classification_report(y_test, y_pred))
print (accuracy_score(y_test, y_pred))

[[1373  526]
 [ 202 3933]]
              precision    recall  f1-score   support

           0       0.87      0.72      0.79      1899
           1       0.88      0.95      0.92      4135

   micro avg       0.88      0.88      0.88      6034
   macro avg       0.88      0.84      0.85      6034
weighted avg       0.88      0.88      0.88      6034

0.8793503480278422


In [38]:
DataTest.head()

,User_ID,Browser_Used,Device_Used,Clean_Description
0,9602,Edge,Desktop,friend stay hotel jacksonvil concert. alway ag...
1,8749,Google Chrome,Mobile,"enjoy stay earli flights. servic good, clean, ..."
2,15500,Chrome,Mobile,stop seattl train tour west great experience. ...
3,5495,Mozilla Firefox,Desktop,stay hotel time busi travel plan weeks. alway ...
4,18570,Edge,Mobile,excel locat hop hop citi transport stop outsid...


In [39]:
basicTest = basictf.transform(DataTest['Clean_Description'])
y_pred = basicModel.predict(basicTest)

In [40]:
DataTest["Predicted_Values"]=y_pred

In [ ]:
DataTest['Predicted_Values'] = DataTest['Predicted_Values'].apply(lambda x:'Bad' if x==0 else 'Good')

In [41]:
DataTest=DataTest.drop(['Browser_Used','Device_Used','Clean_Description'],axis=1)

In [42]:
DataTest.to_csv("Final_Submission.csv",sep="~",index=False)